In [6]:
#@title save online 2-5 data from wandb to json
import wandb
import json

wandb.login(key = "f120e5e4c8c84329e87f496f85e6f7ded7732680")
api = wandb.Api()
runs = api.runs(path="ruisu/clip_count_filtered_images_reviewed")
# num_word_to_number = {
#         "two":2,
#         "three":3,
#         "four":4,
#         "five":5
#     }
numbers = [2,3,4,5]

my_train_set = []
for run in runs:
    run_summary = run.summary
    if run_summary["number"] in numbers:
        if "dataset_name" in run_summary:
            dataset_name = run_summary["dataset_name"]
        else:
            dataset_name = None

        if "org_id" in run_summary:
            org_id = run_summary["org_id"]
        else:
            org_id = None
        sample = {
            "number":run_summary["number"],
            "target":run_summary["target"],
            "target_context":run_summary["target_context"],
            "image_url":run_summary["image_url"],
            "dataset_name":dataset_name,
            "org_id":org_id
        }
        my_train_set.append(sample)
    

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/susu/.netrc


In [8]:
len(my_train_set)

with open("/Users/susu/Library/CloudStorage/OneDrive-Personal/Wisc/clip_count_continued/CLIP_Counting/online_data_0518_4class/online_data_class_2345_2nd_run_need_to_revise.json","w") as f:
    json.dump(my_train_set,f,indent=4)

In [ ]:
# exited_reviewed_file = "/Users/susu/Library/CloudStorage/OneDrive-Personal/Wisc/clip_count_continued/CLIP_Counting/online_data_0513/reviewed_marged_class_2345.json"
# exited_reviewed_file = "/Users/susu/Library/CloudStorage/OneDrive-Personal/Wisc/clip_count_continued/CLIP_Counting/online_data_0518_4class/online_data_class_2345_reviewed.json"
exited_reviewed_file = "/Users/susu/Library/CloudStorage/OneDrive-Personal/Wisc/clip_count_continued/CLIP_Counting/online_data_0518_4class/online_data_class_2345_2nd_run_need_to_filter_no_uploaded.json"
with open(exited_reviewed_file, 'r') as file:
    exited_reviewed_data = json.load(file)
print(len(exited_reviewed_file))
for ele in exited_reviewed_data:
    try:
        img_response = requests.get(ele["image_url"], timeout=10)
        img = Image.open(BytesIO(img_response.content))
    except:
        continue
    
    wandb.init(project="clip_count_filtered_images_reviewed", entity="ruisu")
    wandb.log({
        "number": ele["number"],
        'target': ele["target"],
        'target_context': ele["target_context"],
        "image_url":ele["image_url"],
        "dataset_name":ele["dataset_name"],
        "org_id":ele["org_id"],
        "image":wandb.Image(img, caption=ele["target_context"]),
        "reviewed":True,
    })
    wandb.finish()